# QOSF Screening Tasks
I've included the solutions and my reasoning for each one. The requirement is one task completed, but I wanted to challenge myself a little, so I've done more than one.

## Task 2 - Complex Amplitudes
**Input**: A list or array of four complex amplitudes `[a0, a1, a2, a3]` that define the desired two-qubit state.
   - Ensure that the state is normalized:
     $|a_0|^2 + |a_1|^2 + |a_2|^2 + |a_3|^2 = 1$.
   - If the input is not normalized, include a normalization step.

**Output**: A representation of the two-qubit quantum state vector, for example as a NumPy array:
     $\ket{\psi} = a_0\ket{00} + a_1\ket{01} + a_2\ket{10} + a_3\ket{11}.$
 
 - Do not use quantum-specific state preparation functions from libraries.

**Testing**:
   - Write unit tests that check:
     - Normalization is enforced.
     - The output vector has the correct dimension (4 for two qubits).

**Stretch Goal**: Generalize the implementation to support a three-qubit state given 8 amplitudes.

### Solution
---
The following solution is already generalized to $n$ qubits. The number of amplitudes must be $2^n$. I think this solution is pretty self-explanatory, you just take an array of complex numbers, normalize them if needed, and check the dimensionality.

In [5]:
import numpy as np

def prepare_state(amplitudes):
    state = np.array(amplitudes, dtype=np.complex128)

    # Normalize if sum is not 1
    norm = np.linalg.norm(state)
    if not np.isclose(norm, 1.0):
        state = state / norm

    # Check dimension (must be 2^n)
    n_qubits = int(np.log2(len(state)))
    if 2**n_qubits != len(state):
        raise ValueError("Number of amplitudes must be 2^n for n qubits")
    
    return state

The unit tests are provided below.

In [ ]:
import unittest

class TestPrepareState(unittest.TestCase):
    def test_normalization(self):
        amps = [0, 1+1j, 0, 0] # (1 + 1i)|01>  -> ((1 + 1i) / sqrt(2)) |01>
        state = prepare_state(amps)
        self.assertTrue(np.isclose(np.linalg.norm(state), 1.0))
    
    def test_dimension(self):
        amps = [1, 0, 0, 0] # |00>
        state = prepare_state(amps)
        self.assertEqual(state.shape, (4,))
    
    def test_invalid_dimension(self):
        with self.assertRaises(ValueError):
            prepare_state([1, 0, 0])  # not 2^n

unittest.main(argv=[''], verbosity=2, exit=False)

#### Extended solution
Since I've tried tackling error correction in task 3, I've put together a quick and dirty library called *qlib* to make my life easier. You can check out the source in the qlib folder.

The library focues mainly on density matrices rather than state vectors. A state is initialized similarly to the solution of task 2:

In [10]:
import qlib as ql

two_qubit_state = ql.State(amplitudes=[1, 0, 0, 1])  # \phi^+ bell state
three_qubit_state = ql.State(num_qubits=3) # |000> state

two_qubit_state.probabilities(), three_qubit_state.probabilities()

(array([0.5, 0. , 0. , 0.5]), array([1., 0., 0., 0., 0., 0., 0., 0.]))

Basic circuit functionality is supported via the Circuit class:

In [14]:
import qlib as ql

bell_circuit = ql.Circuit(num_qubits=2)
bell_circuit.add_gate(ql.ops.H, qubit=0)
bell_circuit.CNOT(control_qubit=0, target_qubit=1)

state_00 = ql.State(amplitudes=[1, 0, 0, 0])
state_01 = ql.State(amplitudes=[0, 1, 0, 0])

bell_circuit.run(initial_state=state_00) # produces \phi^+ state
bell_circuit.run(initial_state=state_01) # produces \psi^+ state

state_00.probabilities(), state_01.probabilities()

(array([0.5, 0. , 0. , 0.5]), array([0. , 0.5, 0.5, 0. ]))

Basic noise modelling is also supported via the NoiseModel abstract class:

In [24]:
import qlib as ql

# Assumes a single qubit system
class BitFlipChannel(ql.NoiseModel):
    def __init__(self, p: int):
        super().__init__()
        self.p = p

    def apply(self, state: ql.State):
        rho = state.matrix
        rho_noisy = (1 - self.p) * rho + self.p * (ql.ops.X @ rho @ ql.ops.X)
        state.matrix = rho_noisy

circuit = ql.Circuit(num_qubits=1)
circuit.add_gate(ql.ops.X, qubit=0, noise_model=BitFlipChannel(0.2))   # Noisy X gate

state = ql.State(num_qubits=1)
circuit.run(initial_state=state)

state.probabilities()

array([0.2, 0.8])

Measurement is not fully supported, but expectation values are:

In [26]:
import qlib as ql

state = ql.State(amplitudes=[1, 1])

# <Z> = 0, <X> = 1 because the state is |+>
state.expectation_value(ql.ops.Z), state.expectation_value(ql.ops.X)

(np.complex128(0j), np.complex128(0.9999999999999998+0j))

To see the the library in action, take a peek at Task 3, even though it's not fully finished and won't affect admissions :) Feedback is always appreciated!

## Task 3 - Error Correction
1. Build a function to create a simple noise model. Introduce a random Pauli (X with “a” probability, Z with “b” probability) into any circuit. Test the noise model with simple circuits. 
2. Code the quantum repetition code: https://errorcorrectionzoo.org/c/quantum_repetition. Test the code with the noise model and only X errors. Why does the method not work for Z errors? 
3. Code the Shor code: https://errorcorrectionzoo.org/c/shor_nine. Test the code with your noise model.
4. Code the Hamming code: https://errorcorrectionzoo.org/c/hamming743. Test your code with your noise model. 
5. What are the differences between the Shor and Hamming codes? 
6. What challenges have you detected in the process of building the error-correcting codes? 

### Solution
---
#### 1. Simple Noise Model

Theoretically, this is called a quantum channel. A quantum channel $\mathcal{E}$ maps a state $\rho$ to another state $\rho'$. In this case, states are represented via density matrices (or density operators) which are constructed by taking the outer product of a state: $\rho = \ket{\psi}\bra{\psi}$.

The quantum channel $\mathcal{E}$ for a random Pauli X, also known as the bit-flip channel, is given by:
    $$\mathcal{E}(\rho) = (1-p) \rho + p X \rho X^\dagger, $$
     where $p$ is the probabiliy that a bit-flip occurs. Similarly, the quantum channel for a random Pauli Z, also know as the phase-flip channel, is given by:
    $$\mathcal{E}(\rho) = (1-p) \rho + p Z \rho Z^\dagger. $$
We can generalize this for any Pauli matrix $M$ and call it a Pauli channel:
    $$\mathcal{E}(\rho) = (1-p) \rho + p M \rho M^\dagger. $$

**NOTE**: I've put together a quick and dirty library called *qlib* to make my life easier. Writing it all out in this notebook makes no practical sense. Refer to Task 2 for further explanation. The library comes with an abstract class called *NoiseModel*, which we should inherit and override the apply method:

In [ ]:
import qlib as ql
import numpy as np

class PauliNoiseModel(ql.NoiseModel):
    def __init__(self, a: int, b: int):
        super().__init__()
        self.a = a
        self.b = b
    
    def pauli_channel(self, rho: np.ndarray, p: int, pauli: np.ndarray) -> np.ndarray:
        return (1 - p) * rho + p * (pauli @ rho @ pauli.conj().T)

    def apply(self, state: ql.State):
        state.matrix = self.pauli_channel(state.matrix, self.a, ql.ops.X)
        state.matrix = self.pauli_channel(state.matrix, self.b, ql.ops.Z)

**NOTE**: The question is a little ambiguous to me, so in the case that you meant apply exactly *one* Pauli gate, then the correct channel would be:
   $$ \mathcal{E}(\rho) = (1 - a - b) \rho + a X \rho X + b Z \rho Z, $$
   where $a$ and $b$ are probabilities for a bit-flip and phase-flip, respectively. The noise model is then just:

In [ ]:
class PauliNoiseModel(ql.NoiseModel):
    def __init__(self, a: int, b: int):
        super().__init__()
        
        self.a = a
        self.b = b
        self.prob_sum = a + b

        if self.prob_sum > 1 or self.prob_sum < 0:
            raise Exception("Total probability must be in the range [0, 1]")
    
    def channel(self, rho: np.ndarray) -> np.ndarray:
        return (1 - self.prob_sum) * rho + self.a * (ql.ops.X @ rho @ ql.ops.X) + self.b * (ql.ops.Z @ rho @ ql.ops.Z)

    def apply(self, state: ql.State):
        state.matrix = self.channel(state.matrix)

From this point on, I assume the sequential approach. We shall now generalize this noise model to a system of $n$ qubits. This is a quite forward step; all we have to do is introduce a qubit parameter for the channel function, and then apply this channel to each qubit independently:

In [100]:
import qlib as ql
import numpy as np

class PauliNoiseModel(ql.NoiseModel):
    def __init__(self, a: int, b: int):
        super().__init__()
        self.a = a
        self.b = b
    
    def pauli_channel(self, rho: np.ndarray, qubit: int, p: int, pauli: np.ndarray) -> np.ndarray:
        gate = ql.build_operator(rho, qubit, pauli)
        return (1 - p) * rho + p * (gate @ rho @ gate.conj().T)

    def apply(self, state: ql.State):
        rho_noisy = state.matrix.copy()
        
        # Apply noise to each physical qubit independently
        for qubit in range(state.qubits):
            rho_noisy = self.pauli_channel(rho_noisy, qubit, self.a, ql.ops.X)
            rho_noisy = self.pauli_channel(rho_noisy, qubit, self.b, ql.ops.Z)

        state.matrix = rho_noisy

You might have noticed the `ql.build_operator(rho, qubit, U)` function. This function builds an operator which is only applied to the specified qubit. If we have a 3 qubit system (dimension of $\rho$ is $2^3$), then `U_2 = ql.build_operator(rho, 1, U)` produces the following:
$$ U_2 = I \otimes U \otimes I $$

**NOTE**: It should be obvious that in this noise model, we assume that each qubit might *indepedently* experience a bit-flip with probability $a$ and after that a phase-flip with probability $b$. This plays an important role in the next part.

#### 2. Repetition code

Repetition coding encodes a single qubit $\ket{\psi}$ into $n$ qubits. These qubits together are called a *logical qubit* and are usually denoted by $\ket{\psi_L}$. The general state of a logical qubit can be described by a superposition:

$$ \ket{\psi_L} = \alpha \ket{0_L} + \beta \ket{1_L}. $$

We shall consider the case where $n = 3$ for simplicity. The logical basis states are then given by repeating the computational basis states 3 times:
$$ \ket{0_L} \to \ket{0}^{\otimes 3} = \ket{000}, $$
$$ \ket{1_L} \to \ket{1}^{\otimes 3} = \ket{111}. $$

Error correction is performed in two steps:
  - Eigenvalue measurements of *stabilizers* (I like to call them parity operators)
  - Flipping a qubit based on the measurement results

Stabilizers are a special kind of operator that measure the parity between adjacent qubits. In repetition coding, the set of stabilizers $\mathcal{S}$ for a $n$ logical qubit is:

$$ \mathcal{S} = \{ Z_1 Z_2, \dots, Z_{n - 1} Z_n  \}, $$

where the subscripts indicate which qubit the operator operates on. In our case of $n = 3$, the set becomes:

$$ \mathcal{S} = \{ Z Z I, I Z Z  \}, $$

where $Z Z I$ is shorthand notation for $Z \otimes Z \otimes I$, and so on. These two stabilizers have the following measurement outcomes:
  - $ZZI$ returns +1 if qubits 1 and 2 have the same value, -1 if different
  - $IZZ$ returns +1 if qubits 2 and 3 have the same value, -1 if different
  - **NOTE**: In other words, +1 and -1 are the eigenvalues of stabilizers

The following table provides the correction step based on the outcome of measuring ZZI and IZZ.
| Error     | State after error                      | ZZI | IZZ | Correction |
| --------  | -------------------------------------  | -- | -- | ------------ |
| None      | $\alpha \ket{000} + \beta \ket{111}$   | +1 | +1 | None         |
| Qubit 1   | $\alpha \ket{100} + \beta \ket{011}$   | -1 | +1 | Flip qubit 1 |
| Qubit 2   | $\alpha \ket{010} + \beta \ket{101}$   | -1 | -1 | Flip qubit 2 | 
| Qubit 3   | $\alpha \ket{001} + \beta \ket{110}$   | +1 | -1 | Flip qubit 3 |

For phase-flip (Z) errors:
   - $Z_1|\psi_L\rangle = \alpha|000\rangle - \beta|111\rangle$
   - $Z_2|\psi_L\rangle = \alpha|000\rangle - \beta|111\rangle$
   - $Z_3|\psi_L\rangle = \alpha|000\rangle - \beta|111\rangle$

All Z errors produce the same state (differing only by a global phase), so the stabilizers cannot distinguish between them.

In [101]:
import qlib as ql
import numpy as np

def repetition_code_correction(state: ql.State):
    num_qubits = state.qubits
    if num_qubits != 3:
        raise ValueError("Repetition code requires 3 qubits")
    
    # Stabilizers
    ZZI = ql.build_operators(state.matrix, [0, 1], [ql.ops.Z, ql.ops.Z])
    IZZ = ql.build_operators(state.matrix, [1, 2], [ql.ops.Z, ql.ops.Z])
    
    # Measurement
    # Expectation value gives the average value, so it's not going to be +1 or -1 exactly
    synd1 = state.expectation_value(ZZI)
    synd2 = state.expectation_value(IZZ)

    correction_applied = "None"
    if synd1 < 0 and synd2 > 0:  # (-1, +1)
        state.apply_gate(ql.ops.X, (0,))
        correction_applied = "X on qubit 0"
    elif synd1 < 0 and synd2 < 0:  # (-1, -1)  
        state.apply_gate(ql.ops.X, (1,))
        correction_applied = "X on qubit 1"
    elif synd1 > 0 and synd2 < 0:  # (+1, -1)
        state.apply_gate(ql.ops.X, (2,))
        correction_applied = "X on qubit 2"

    return synd1, synd2, correction_applied

The following is a *deterministic error* (noiseless) test of the correction function with a single bit flip. The initial logical state $\ket{\psi_L}$ is given by an equal superposition:

$$ \ket{\psi_L} = \frac{1}{\sqrt{2}} ( \ket{0_L} + \ket{1_L} ) = \frac{1}{\sqrt{2}} ( \ket{000} + \ket{111} ). $$

Every bit is flipped one by one, corrected and then the results are documented.

In [112]:
import pandas as pd
from IPython.display import display, HTML

states_error = []
states_corrected = []
syndromes = []

for i in range(3):
    original_state = ql.State(amplitudes=[1, 0, 0, 0, 0, 0, 0, 1])  # 1/sqrt(2)(|0_L> + |1_L>) = 1/sqrt(2)(|000> + |111>)
    
    circuit = ql.Circuit(num_qubits=3)
    circuit.add_gate(ql.ops.X, qubit=i) # Apply a specific X error to qubit 0
    state = circuit.run(initial_state=original_state)
    temp_for_table = []
    
    for i, prob in enumerate(state.matrix.diagonal()):
        if abs(prob) > 0.001:
            temp_for_table.append(f"P({i:03b}) = {np.abs(prob):.3f}")

    states_error.append('\n'.join(temp_for_table))
    (synd1, synd2, corr) = repetition_code_correction(state)

    syndromes.append((synd1, synd2, corr))

    temp_for_table.clear()
    for i, prob in enumerate(state.matrix.diagonal()):
        if abs(prob) > 0.001:
            temp_for_table.append(f"P({i:03b}) = {np.abs(prob):.3f}")

    states_corrected.append('\n'.join(temp_for_table))

data = {
    "Error": ["Qubit 0", "Qubit 1", "Qubit 2"],
    "State after error": states_error,
    "ZZI": [v[0].real for v in syndromes],
    "IZZ": [v[1].real for v in syndromes],
    "Correction": [v[2] for v in syndromes],
    "State after correction": states_corrected
}

df = pd.DataFrame(data)
display(HTML("<center>" + df.to_html().replace("\\n","<br>") + "</center>")) # I don't understand why pandas doesn't just allow new lines...

,Error,State after error,ZZI,IZZ,Correction,State after correction
0,Qubit 0,P(011) = 0.500P(100) = 0.500,-1.0,1.0,X on qubit 0,P(000) = 0.500P(111) = 0.500
1,Qubit 1,P(010) = 0.500P(101) = 0.500,-1.0,-1.0,X on qubit 1,P(000) = 0.500P(111) = 0.500
2,Qubit 2,P(001) = 0.500P(110) = 0.500,1.0,-1.0,X on qubit 2,P(000) = 0.500P(111) = 0.500


The following is the same test but with the noise model from step 1. The probabiliy of a bit-flip occuring for each qubit is $a = 0.8$.

In [103]:
import pandas as pd
from IPython.display import display, HTML

states_error = []
states_corrected = []
syndromes = []

for i in range(3):
    original_state = ql.State(amplitudes=[1, 0, 0, 0, 0, 0, 0, 1])  # 1/sqrt(2)(|0_L> + |1_L>) = 1/sqrt(2)(|000> + |111>)
    
    circuit = ql.Circuit(num_qubits=3)
    circuit.add_gate(ql.ops.X, qubit=i) # Apply a specific X error to qubit 0
    state = circuit.run(initial_state=original_state, noise_model=PauliNoiseModel(0.8, 0))
    temp_for_table = []
    
    for i, prob in enumerate(state.matrix.diagonal()):
        if abs(prob) > 0.001:
            temp_for_table.append(f"P({i:03b}) = {np.abs(prob):.3f}")

    states_error.append('\n'.join(temp_for_table))
    (synd1, synd2, corr) = repetition_code_correction(state)

    syndromes.append((synd1, synd2, corr))

    temp_for_table.clear()
    for i, prob in enumerate(state.matrix.diagonal()):
        if abs(prob) > 0.001:
            temp_for_table.append(f"P({i:03b}) = {np.abs(prob):.3f}")

    states_corrected.append('\n'.join(temp_for_table))

data = {
    "Error": ["Qubit 0", "Qubit 1", "Qubit 2"],
    "State after error": states_error,
    "ZZI": [v[0].real for v in syndromes],
    "IZZ": [v[1].real for v in syndromes],
    "Correction": [v[2] for v in syndromes],
    "State after correction": states_corrected
}

df = pd.DataFrame(data)
display(HTML("<center>" + df.to_html().replace("\\n","<br>") + "</center>")) # I don't understand why pandas doesn't just allow new lines...

,Error,State after error,ZZI,IZZ,Correction,State after correction
0,Qubit 0,P(000) = 0.080P(001) = 0.080P(010) = 0.080P(011) = 0.260P(100) = 0.260P(101) = 0.080P(110) = 0.080P(111) = 0.080,-0.36,0.36,X on qubit 0,P(000) = 0.260P(001) = 0.080P(010) = 0.080P(011) = 0.080P(100) = 0.080P(101) = 0.080P(110) = 0.080P(111) = 0.260
1,Qubit 1,P(000) = 0.080P(001) = 0.080P(010) = 0.260P(011) = 0.080P(100) = 0.080P(101) = 0.260P(110) = 0.080P(111) = 0.080,-0.36,-0.36,X on qubit 1,P(000) = 0.260P(001) = 0.080P(010) = 0.080P(011) = 0.080P(100) = 0.080P(101) = 0.080P(110) = 0.080P(111) = 0.260
2,Qubit 2,P(000) = 0.080P(001) = 0.260P(010) = 0.080P(011) = 0.080P(100) = 0.080P(101) = 0.080P(110) = 0.260P(111) = 0.080,0.36,-0.36,X on qubit 2,P(000) = 0.260P(001) = 0.080P(010) = 0.080P(011) = 0.080P(100) = 0.080P(101) = 0.080P(110) = 0.080P(111) = 0.260


We can see that after correction, the states $\ket{000} = \ket{0_L}$ and $\ket{111} = \ket{1_L}$ stand out from the rest. The rest have probabilty of being measured due to:

- Residual uncorrected errors
- Multiple error combinations

This happens because we introduced noise into the system, but the trend is clearly toward correction.

Let's discuss practical implementation for a moment. To create the logical qubit state, we would use two CNOT gates to entangle two additional repeated qubits. Then, any unitary we want to perform on the logical qubit, we have to perform for all of the qubits: $U^{\otimes 3} = U \otimes U \otimes U$. In the circuit shown, $U$ represents a circuit operating on the logical qubit and possibly some noise.

<center><img src="img/repeatingcodeimpl.png"/></center>

After the circuit $U$ has been performed, we peform error correction. We introduce two ancilla qubits, which are used for stabilizer measurements. The first pair of CNOTs represent the stabilizer $ZZI$ and the second pair $IZZ$. We then measure these two ancilla qubits, and we pass the results (+1 being classical bit 0, and -1 being classical bit 1) to a field programmable gate array or microcontroller which then decides which qubits to flip.